### Import and check data

In [1]:
import pandas as pd

In [10]:
df_aisles = pd.read_csv('Data/aisles.csv')
df_departments = pd.read_csv('Data/departments.csv')
df_oder_products = pd.read_csv('Data/oder_products_s.csv')
df_oders = pd.read_csv('Data/oders_s.csv')
df_products = pd.read_csv('Data/products.csv')
df_tips = pd.read_csv('Data/tips.csv')
dfs = [df_aisles, df_departments, df_oder_products, df_oders, df_products, df_tips]

In [3]:
for df in dfs:
    print(df.dtypes)  # Ok

aisle_id     int64
aisle       object
dtype: object
department_id     int64
department       object
dtype: object
order_id             int64
product_id           int64
add_to_cart_order    int64
dtype: object
order_id                    int64
user_id                     int64
order_number                int64
order_dow                   int64
order_hour_of_day           int64
days_since_prior_order    float64
dtype: object
product_id        int64
product_name     object
aisle_id          int64
department_id     int64
dtype: object
Unnamed: 0    int64
order_id      int64
tip            bool
dtype: object


In [4]:
for df in dfs:
    print(df.info())  # Missing Data only in 'days_since_prior_order': first order of the user -> OK

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   aisle_id  134 non-null    int64 
 1   aisle     134 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   department_id  21 non-null     int64 
 1   department     21 non-null     object
dtypes: int64(1), object(1)
memory usage: 468.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16942236 entries, 0 to 16942235
Data columns (total 3 columns):
 #   Column             Dtype
---  ------             -----
 0   order_id           int64
 1   product_id         int64
 2   add_to_cart_order  int64
dtypes: int64(3)
memory usage: 387.8 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeInd

In [5]:
# Check if IDs are needed (same name for different IDs?)
df_aisles[df_aisles['aisle'].duplicated()==True]  # no ID has the same name -> aisle_id can be removed

,aisle_id,aisle


In [6]:
df_departments[df_departments['department'].duplicated()==True]  # no ID has the same name -> department_id can be removed

,department_id,department


In [7]:
df_products[df_products['product_name'].duplicated()==True]  # no ID has the same name -> product_id can be removed

,product_id,product_name,aisle_id,department_id


### Merge dataframes for analytics
df_orders: orders with tip <br>
df_order_products: ordered products with informations of the order, department and aisle.

In [8]:
df_orders = df_oders.merge(df_tips, on='order_id').drop(['Unnamed: 0'], axis=1)
df_orders

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip
0,2539329,1,1,2,8,NaN,False
1,2398795,1,2,3,7,15.0,False
2,473747,1,3,3,12,21.0,False
3,2254736,1,4,4,7,29.0,False
4,431534,1,5,4,15,28.0,False
...,...,...,...,...,...,...,...
1673016,2266710,206209,10,5,18,29.0,False
1673017,1854736,206209,11,4,10,30.0,False
1673018,626363,206209,12,1,12,18.0,False
1673019,2977660,206209,13,1,12,7.0,False


In [11]:
df_order_products = df_oder_products.merge(df_oders, on='order_id').merge(df_products, on='product_id').merge(df_departments, on='department_id').merge(df_aisles, on='aisle_id').drop(['product_id', 'aisle_id', 'department_id'], axis=1)
df_order_products

,order_id,add_to_cart_order,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,department,aisle
0,2,1,202279,3,5,9,8.0,Organic Egg Whites,dairy eggs,eggs
1,2,2,202279,3,5,9,8.0,Michigan Organic Kale,produce,fresh vegetables
2,2,3,202279,3,5,9,8.0,Garlic Powder,pantry,spices seasonings
3,2,4,202279,3,5,9,8.0,Coconut Butter,pantry,oils vinegars
4,2,5,202279,3,5,9,8.0,Natural Sweetener,pantry,baking ingredients
...,...,...,...,...,...,...,...,...,...,...
16942231,3421058,4,136952,20,3,18,15.0,Club Soda Lower Sodium,beverages,soft drinks
16942232,3421058,5,136952,20,3,18,15.0,Classic Britannia Crisps,snacks,crackers
16942233,3421058,6,136952,20,3,18,15.0,Baby Brie,dairy eggs,packaged cheese
16942234,3421058,7,136952,20,3,18,15.0,Genoa Salame with White Cheddar Cheese & Toast...,meat seafood,packaged meat
